In [30]:
import requests
import bs4
import xml.etree.ElementTree as ET
import os
import re

In [46]:
def get_IC_names(filename):
    root = ET.parse(filename).getroot()
    elements_info = root.findall(".//elements/element")
    ic_names = {}
    for element in elements_info:
        element_name = element.get('name')
        result = re.search('^[U][0-9]+', element_name)
        if result is not None:
            ic_names[int(element_name[1:])] = element.get('value')
    print(ic_names)
    return [ic_names[i+1] for i in range(len(ic_names))]

In [48]:
get_IC_names(os.path.abspath("brd-parser/data/usbhost.brd"))

{2: 'RT9080-3.3', 3: 'MCP73831T-2ACI/OT', 1: '8MB QSPI'}


['8MB QSPI', 'RT9080-3.3', 'MCP73831T-2ACI/OT']

In [28]:
filename = os.path.abspath("brd-parser/data/featherwing.brd")
search_term = get_IC_names(filename)
url = 'https://google.com/search?q=' + search_term + ' power consumption'
request_result=requests.get(url)
soup = bs4.BeautifulSoup(request_result.text, "lxml")

In [32]:
first_relevant_result = None
for c in soup.find_all("h3"):
    if search_term.lower() in c.text.lower():
        first_relevant_result = c
        break

In [37]:
first_link = first_relevant_result.parent
while "href" not in first_link.attrs and first_link is not soup:
    first_link = first_link.parent

[PDF] FGPMMOPA6H GPS Standalone Module Data Sheetcdn-shop.adafruit.com › GlobalTop-FGPMMOPA6H-Datasheet-V0A
[PDF] FGPMMOPA6H GPS Standalone Module Data Sheetcdn-shop.adafruit.com › GlobalTop-FGPMMOPA6H-Datasheet-V0A


In [38]:
first_link

<a href="/url?q=https://cdn-shop.adafruit.com/datasheets/GlobalTop-FGPMMOPA6H-Datasheet-V0A.pdf&amp;sa=U&amp;ved=2ahUKEwjR1YCH2Ob-AhXBOn0KHfSHBUsQFnoECAYQAg&amp;usg=AOvVaw1B7XpzjhRpHr5bCewJKpYP"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div class="BNeawe vvjwJb AP7Wnd">[PDF] FGPMMOPA6H GPS Standalone Module Data Sheet</div></h3></div><div class="sCuL3"><div class="BNeawe UPmit AP7Wnd lRVwie">cdn-shop.adafruit.com › GlobalTop-FGPMMOPA6H-Datasheet-V0A</div></div></div></a>

In [41]:
first_link.parent.next_sibling.text

'Jan 31, 2012 · FGPMMOPA6H is excellent low power consumption characteristic (acquisition 82mW, tracking. 66mW), power sensitive devices, especially portable\xa0...'

In [44]:
words = first_link.parent.next_sibling.text.rstrip().split()
for i in range(len(words)):
    if "mw" in words[i].lower():
        print(words[i-1], words[i])

(acquisition 82mW,
tracking. 66mW),


In [50]:
def extract_mw_info(filename):
    search_terms = [filename]
    if filename.endswith(".brd"):
        search_terms = get_IC_names(filename)
    for search_term in search_terms:
        print("[INFO] Searching for power consumption of " + search_term)
        url = 'https://google.com/search?q=' + search_term + ' power consumption'
        request_result=requests.get(url)
        soup = bs4.BeautifulSoup(request_result.text, "lxml")
        first_relevant_result = None
        found_info = False
        for c in soup.find_all("h3"):
            # if search_term.lower() in c.text.lower():
            first_relevant_result = c
            first_link = first_relevant_result.parent
            while "href" not in first_link.attrs and first_link is not soup:
                first_link = first_link.parent
            print(first_link)
            htmltext = first_link.parent.next_sibling.text
            print(htmltext)
            words = htmltext.rstrip().split()
            results = []
            for i in range(len(words)):
                if "mw" in words[i].lower():
                    print(words[i-1], words[i])
                    results.append((words[i-1], words[i]))
            if len(results) > 0:
                found_info = True
                break
        if not found_info:
            print("[WARNING] No power consumption information found for " + search_term)

In [52]:
filename = os.path.abspath("brd-parser/data/featherwing.brd")
extract_mw_info(filename)

{1: 'FGPMMOPA6H'}
[INFO] Searching for power consumption of FGPMMOPA6H
<a href="/url?q=https://cdn-shop.adafruit.com/datasheets/GlobalTop-FGPMMOPA6H-Datasheet-V0A.pdf&amp;sa=U&amp;ved=2ahUKEwj-osv-5ub-AhXMCjQIHfzaDW4QFnoECAoQAg&amp;usg=AOvVaw3ntOmh_ths24LWAKzo4jJ7"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div class="BNeawe vvjwJb AP7Wnd">[PDF] FGPMMOPA6H GPS Standalone Module Data Sheet</div></h3></div><div class="sCuL3"><div class="BNeawe UPmit AP7Wnd lRVwie">cdn-shop.adafruit.com › GlobalTop-FGPMMOPA6H-Datasheet-V0A</div></div></div></a>
Jan 31, 2012 · FGPMMOPA6H is excellent low power consumption characteristic (acquisition 82mW, tracking. 66mW), power sensitive devices, especially portable ...
(acquisition 82mW,
tracking. 66mW),


In [53]:
ICname = "FAN3852"
extract_mw_info(ICname)

[INFO] Searching for power consumption of FAN3852
<a href="/url?q=https://www.onsemi.com/pdf/datasheet/fan3852-d.pdf&amp;sa=U&amp;ved=2ahUKEwjav_yB5-b-AhWVAzQIHTL_AEcQFnoECAMQAg&amp;usg=AOvVaw2ugRC9E5_7OzBIrLguEQJz"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div class="BNeawe vvjwJb AP7Wnd">[PDF] FAN3852 - Microphone Pre-Amplifier with Digital Output - Onsemi</div></h3></div><div class="sCuL3"><div class="BNeawe UPmit AP7Wnd lRVwie">www.onsemi.com › pdf › datasheet › fan3852-d</div></div></div></a>
The FAN3852 features an integrated LDO and is powered from the system supply rails up to 3.63 V, with low power consumption of only.
<a href="/url?q=https://www.digikey.co.il/htmldatasheets/production/3618101/0/0/1/fan3852.html&amp;sa=U&amp;ved=2ahUKEwjav_yB5-b-AhWVAzQIHTL_AEcQFnoECAkQAg&amp;usg=AOvVaw3ahVyq6Dt4XPCvGDyYxzOK"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div class="BNeawe vvjwJb AP7Wnd">FAN3852 Datasheet by ON Semiconductor - Digi-Ke

In [54]:
ICname = "ACS732"
extract_mw_info(ICname)

[INFO] Searching for power consumption of ACS732
<a href="/url?q=https://www.allegromicro.com/-/media/files/datasheets/acs732-3-kla-datasheet.pdf&amp;sa=U&amp;ved=2ahUKEwjFleyD5-b-AhX9GDQIHZfbAXMQFnoECAkQAg&amp;usg=AOvVaw2GI_5bqaRbZX1x7C83ct0T"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div class="BNeawe vvjwJb AP7Wnd">[PDF] ACS732 and ACS733 - Allegro MicroSystems</div></h3></div><div class="sCuL3"><div class="BNeawe UPmit AP7Wnd lRVwie">www.allegromicro.com › files › datasheets › acs732-3-kla-datasheet</div></div></div></a>
Performance values include the power consumed by the PCB. ... ACS732. 4.75. 5.0. 5.25. V. ACS733. 3.14. 3.3. 3.46. V. Supply Current.
<a href="/url?q=https://www.allegromicro.com/en/products/sense/current-sensor-ics/zero-to-fifty-amp-integrated-conductor-sensor-ics/acs732-3&amp;sa=U&amp;ved=2ahUKEwjFleyD5-b-AhX9GDQIHZfbAXMQFnoECAgQAg&amp;usg=AOvVaw27jV4nNsrsx33-FNYjehMD"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div c

In [56]:
extract_mw_info(os.path.abspath("brd-parser/data/usbhost.brd"))

{2: 'RT9080-3.3', 3: 'MCP73831T-2ACI/OT', 1: '8MB QSPI'}
[INFO] Searching for power consumption of 8MB QSPI
<a href="/url?q=https://www.adafruit.com/product/5671&amp;sa=U&amp;ved=2ahUKEwi1mNaT5-b-AhUCAjQIHbvgDqIQFnoECAEQAg&amp;usg=AOvVaw3GVQ2-z64x8jPNqjLBlwHc"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div class="BNeawe vvjwJb AP7Wnd">ESP32-C6-WROOM-1-N8 Engineering Module - 8 MB Quad SPI Flash</div></h3></div><div class="sCuL3"><div class="BNeawe UPmit AP7Wnd lRVwie">www.adafruit.com › product</div></div></div></a>
$3.50 Out of stockWith low-power consumption, it is an ideal choice for a variety of IoT devices!This module comes with 8 MB Quad SPI Flash memory.Features:32-bit RISC-V.
<a href="/url?q=https://www.adafruit.com/product/5678&amp;sa=U&amp;ved=2ahUKEwi1mNaT5-b-AhUCAjQIHbvgDqIQFnoECAkQAg&amp;usg=AOvVaw0260sCC6bdj_YXd2FV4iLT"><div class="DnJfK"><div class="j039Wc"><h3 class="zBAuLc l97dzf"><div class="BNeawe vvjwJb AP7Wnd">3V 64Mbit / 8 MB QSPI NOR Flash 